In [1]:
!pip install requests
!pip install bs4
!pip install datasets
!pip install tqdm

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import pandas as pd
file_path = 'C:\\Users\\88697\\Downloads\\ESG_Transcripts.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
len(lines)
df = pd.DataFrame(lines, columns=['text'])
df

In [1]:
!pip install transformers==4.28.0

In [1]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments, BertForMaskedLM
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import pipeline
from transformers import AutoModelForMaskedLM,AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
import os
import math

In [2]:
# check version
import transformers
torch.__version__, transformers.__version__

('2.2.2+cu121', '4.28.0')

In [3]:
torch.cuda.is_available()

True

In [4]:
from datasets import load_dataset

from transformers import AutoModel

from transformers import AutoConfig, AutoModelForSequenceClassification


# pick the model type
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

print(len(tokenizer))


id2label = {
    "0": "Neutral",
    "1": "Positive",
    "2": "Negative"
  }

label2id = {
    "Neutral": 0,
    "Positive": 1,
    "Negative": 2
  }

config = AutoConfig.from_pretrained('yiyanghkust/finbert-pretrain', label2id=label2id, id2label=id2label)

model = BertForMaskedLM.from_pretrained('yiyanghkust/finbert-pretrain', config=config)

30873


Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model_name = "yiyanghkust/finbert-pretrain"
train_file = "C:\\Users\\ASUS\\Downloads\\ESG\\ESG_Transcripts.txt"
eval_file = "C:\\Users\\ASUS\\Downloads\\ESG\\ESG_Transcripts.txt"
max_seq_length = 128
train_epoches = 30
batch_size = 64
out_model_path = "further_pretrain_FinBERT"

In [10]:
from typing import List, Dict
from transformers import DataCollatorForLanguageModeling
from transformers import TrainerCallback, TrainerState, TrainerControl
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

class CustomDataCollatorForLanguageModeling(DataCollatorForLanguageModeling):
    def __call__(self, examples: List[torch.Tensor]) -> Dict[str, torch.Tensor]:
        # 正确地调用父类的 __call__ 方法
        batch = super().__call__(examples)
        labels = batch['input_ids'].clone()
        labels[batch['input_ids'] != self.tokenizer.mask_token_id] = -100
        batch['labels'] = labels
        
        # 打印批次大小
        print(f"Input IDs size: {batch['input_ids'].size()}")
        print(f"Attention Mask size: {batch['attention_mask'].size()}")
        print(f"Labels size: {batch['labels'].size()}")
        return batch


class LoggingCallback(TrainerCallback):
    """A custom callback that logs loss and evaluation results to a CSV file."""
    
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.epoch = 0
        self.metrics = []

    def on_epoch_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        self.epoch += 1
        log = {'epoch': self.epoch}
        # Assuming 'loss' and 'eval_loss' are provided by kwargs when 'on_epoch_end' is called
        log.update({
            'training_loss': state.log_history[-1]['loss'],
            'validation_loss': kwargs.get('metrics', {}).get('eval_loss', None)
        })
        self.metrics.append(log)
        pd.DataFrame(self.metrics).to_csv(self.csv_file, index=False)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# 初始化你的自定義回調
log_callback = LoggingCallback(csv_file='training_log.csv')
# 這裡將`checkpoint-90840`替換成您截圖中的最後檢查點資料夾名
checkpoint_path = r'C:\Users\ASUS\Downloads\ESG\further_pretrain_FinBERT\checkpoint-70243'

# 現在再次嘗試加載分詞器
if os.path.exists(checkpoint_path):
    print(f"Loading model and tokenizer from checkpoint: {checkpoint_path}")
else:
    raise ValueError(f"Checkpoint '{checkpoint_path}' not found.")

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=128,
)

eval_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=eval_file,
    block_size=128,
)

training_args = TrainingArguments(
        output_dir=out_model_path,
        overwrite_output_dir=True,
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        num_train_epochs=train_epoches,
        per_device_train_batch_size=batch_size,
        save_steps=2000,
        save_total_limit=2,
        prediction_loss_only=True,
        load_best_model_at_end=True,
        dataloader_num_workers=4,   # 设置 DataLoader 的工作线程数
        dataloader_pin_memory=True, # 使用 pinned memory，加速 CPU 到 GPU 的数据传
        # 確保設定這個參數，以從中斷點恢復訓練進度
        resume_from_checkpoint=checkpoint_path
    )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    callbacks=[log_callback],
)
torch.cuda.empty_cache()
trainer.train(resume_from_checkpoint=checkpoint_path)

trainer.save_model('further_pretrain_FinBERT/')
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Loading model and tokenizer from checkpoint: C:\Users\ASUS\Downloads\ESG\further_pretrain_FinBERT\checkpoint-70243


c:\Users\ASUS\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
c:\Users\ASUS\anaconda3\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0it [00:00, ?it/s]

  0%|          | 0/110910 [00:00<?, ?it/s]

{'loss': 1.3224, 'learning_rate': 1.8217473627265352e-05, 'epoch': 19.07}
{'loss': 1.3302, 'learning_rate': 1.7992065638806243e-05, 'epoch': 19.2}
{'loss': 1.3352, 'learning_rate': 1.776665765034713e-05, 'epoch': 19.34}
{'loss': 1.3153, 'learning_rate': 1.754124966188802e-05, 'epoch': 19.48}
{'loss': 1.3335, 'learning_rate': 1.7315841673428906e-05, 'epoch': 19.61}
{'loss': 1.3344, 'learning_rate': 1.7090433684969796e-05, 'epoch': 19.75}
{'loss': 1.3312, 'learning_rate': 1.6865025696510687e-05, 'epoch': 19.88}


...

{'eval_loss': 1.137832760810852, 'eval_runtime': 2222.7038, 'eval_samples_per_second': 106.446, 'eval_steps_per_second': 13.306, 'epoch': 29.0}
{'loss': 1.2361, 'learning_rate': 1.537282481291137e-06, 'epoch': 29.08}
{'loss': 1.2439, 'learning_rate': 1.311874492832026e-06, 'epoch': 29.21}
{'loss': 1.2433, 'learning_rate': 1.086466504372915e-06, 'epoch': 29.35}
{'loss': 1.2356, 'learning_rate': 8.61058515913804e-07, 'epoch': 29.48}
{'loss': 1.2418, 'learning_rate': 6.35650527454693e-07, 'epoch': 29.62}
{'loss': 1.2384, 'learning_rate': 4.1024253899558203e-07, 'epoch': 29.75}
{'loss': 1.2307, 'learning_rate': 1.8483455053647103e-07, 'epoch': 29.89}


  0%|          | 0/29575 [00:00<?, ?it/s]

{'eval_loss': 1.143845796585083, 'eval_runtime': 2203.8823, 'eval_samples_per_second': 107.356, 'eval_steps_per_second': 13.42, 'epoch': 30.0}
{'train_runtime': 395497.5724, 'train_samples_per_second': 17.947, 'train_steps_per_second': 0.28, 'train_loss': 0.46821361511136916, 'epoch': 30.0}


  0%|          | 0/29575 [00:00<?, ?it/s]

Perplexity: 3.13


In [12]:
FineTune_df = pd.read_csv('C:\\Users\\ASUS\\Downloads\\Sentences_75Agree.txt', encoding= 'unicode_escape') ## use customized dataset
FineTune_df

,sentence,label
0,"According to Gran , the company has no plans t...",0
1,With the new production plant the company woul...,1
2,"For the last quarter of 2010 , Componenta 's n...",1
3,"In the third quarter of 2010 , net sales incre...",1
4,Operating profit rose to EUR 13.1 mn from EUR ...,1
...,...,...
3375,Operating result for the 12-month period decre...,2
3376,HELSINKI Thomson Financial - Shares in Cargote...,2
3377,LONDON MarketWatch -- Share prices ended lower...,2
3378,Operating profit fell to EUR 35.4 mn from EUR ...,2


In [13]:
list(FineTune_df.columns)

['sentence', 'label']

In [14]:
df_train, df_test, = train_test_split(FineTune_df, stratify=FineTune_df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(2737, 2) (338, 2) (305, 2)


In [5]:
model = BertForSequenceClassification.from_pretrained('further_pretrain_FinBERT/', num_labels=3)

Some weights of the model checkpoint at further_pretrain_FinBERT/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at further_pretrain_Fi

In [16]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
print(dataset_train.shape, dataset_val.shape, dataset_test.shape)

Map:   0%|          | 0/2737 [00:00<?, ? examples/s]

Map:   0%|          | 0/305 [00:00<?, ? examples/s]

Map:   0%|          | 0/338 [00:00<?, ? examples/s]

(2737, 6) (305, 6) (338, 6)


In [6]:

from transformers import TrainerCallback, TrainerState, TrainerControl
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm

In [18]:
class SaveMetricsCallback(TrainerCallback):
    def __init__(self):
        # 初始化一個 DataFrame 來保存度量數據
        self.metrics_df = pd.DataFrame()

    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        metrics = kwargs.get("metrics", {})
        metrics['epoch'] = state.epoch  # 加入 epoch 資訊到度量數據中
        # 創建一個新的 DataFrame 來保存當前的度量數據
        current_metrics_df = pd.DataFrame([metrics])
        # 使用 concat 來合併當前的度量數據
        self.metrics_df = pd.concat([self.metrics_df, current_metrics_df], ignore_index=True)
        # 每次評估後保存 DataFrame 到 CSV
        self.metrics_df.to_csv('metrics.csv', index=False)

# 你的 compute_metrics 函數看起來是正確的，所以我們不需要更改它
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels.flatten(), predictions.flatten(), average='weighted', zero_division=0)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        logging_strategy='no',  # 禁用進度條
)

trainer = Trainer(
        model=model,                 # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,          # evaluation dataset
        compute_metrics=compute_metrics,
        callbacks=[SaveMetricsCallback],
)
no_deprecation_warning=True
trainer.train()

c:\Users\ASUS\anaconda3\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/430 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.2978377640247345, 'eval_accuracy': 0.8918032786885246, 'eval_f1': 0.8929993985361221, 'eval_precision': 0.8986506790382247, 'eval_recall': 0.8918032786885246, 'eval_runtime': 8.6554, 'eval_samples_per_second': 35.238, 'eval_steps_per_second': 1.155, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.35313376784324646, 'eval_accuracy': 0.9114754098360656, 'eval_f1': 0.9111059848349204, 'eval_precision': 0.9109765962036238, 'eval_recall': 0.9114754098360656, 'eval_runtime': 8.6269, 'eval_samples_per_second': 35.355, 'eval_steps_per_second': 1.159, 'epoch': 4.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.3575517237186432, 'eval_accuracy': 0.921311475409836, 'eval_f1': 0.9213869006737626, 'eval_precision': 0.9215823310812267, 'eval_recall': 0.921311475409836, 'eval_runtime': 9.7944, 'eval_samples_per_second': 31.14, 'eval_steps_per_second': 1.021, 'epoch': 5.0}
{'train_runtime': 693.7488, 'train_samples_per_second': 19.726, 'train_steps_per_second': 0.62, 'train_loss': 0.14035823733307595, 'epoch': 5.0}


TrainOutput(global_step=430, training_loss=0.14035823733307595, metrics={'train_runtime': 693.7488, 'train_samples_per_second': 19.726, 'train_steps_per_second': 0.62, 'train_loss': 0.14035823733307595, 'epoch': 5.0})

In [19]:
model.eval()
trainer.predict(dataset_test).metrics

  0%|          | 0/11 [00:00<?, ?it/s]

{'test_loss': 0.21398162841796875,
 'test_accuracy': 0.9319526627218935,
 'test_f1': 0.9319319996243074,
 'test_precision': 0.9319593867670791,
 'test_recall': 0.9319526627218935,
 'test_runtime': 11.0172,
 'test_samples_per_second': 30.679,
 'test_steps_per_second': 0.998}

In [21]:
trainer.save_model('C:\\Users\\ASUS\\Downloads\\ESG\\ESG_Weight')

In [7]:
m = AutoModelForSequenceClassification.from_pretrained("C:\\Users\\ASUS\\Downloads\\ESG\\ESG_Weight")
print(m.config.id2label)

{0: 'Neutral', 1: 'Positive', 2: 'Negative'}


In [8]:
finbert = BertForSequenceClassification.from_pretrained('C:\\Users\\ASUS\\Downloads\\ESG\\ESG_Weight',num_labels=3)

In [48]:
import pandas as pd

# 檔案所在的目錄
directory = 'C:\\Users\\ASUS\\Downloads\\news label v1\\news label v1\\'

# 使用迴圈來生成檔案路徑列表
file_paths = [directory + f'{i}.xlsx' for i in range(1, 147)]

# 讀取所有 CSV 文件並合併成一個 DataFrame
data_frames = [pd.read_excel(file, index_col=None) for file in file_paths]
combined_df = pd.concat(data_frames, ignore_index=True)

# 可以選擇保存合併後的 DataFrame 為新的 CSV 文件
combined_df.to_excel('C:\\Users\\ASUS\\Downloads\\news label v1\\news label v1\\Total_data.xlsx', index=False)

# 打印結果查看
print(combined_df)

      number                                           sentence  \
0          1  12 U.S. Banks and Tech Giants Targeted with Me...   
1          2  During an international news event today with ...   
2          3  Arjuna Capitals median gender pay gap proposal...   
3          4  In January, Citigroup surprised investors when...   
4          5  In doing so, Citi became the first U.S. compan...   
...      ...                                                ...   
2972       5  "Climate action tops the list of our sustainab...   
2973       6  The goals we set are truly ambitious and will ...   
2974       7  "However, we believe this is the right thing t...   
2975       8  While we will carefully evaluate our options f...   
2976       9  Yum China will continue to enhance transparenc...   

      researcher label  ChatGPT label  researcher2 label  final label  \
0                    0              0                NaN          0.0   
1                    0              2            

In [50]:
file_path = 'C:\\Users\\ASUS\\Downloads\\news label v1\\news label v1\\Total_data.xlsx'
data = pd.read_excel(file_path)

# 定义将数字标签转换为情绪标签的函数
def convert_label(item):
    label_map = {'0': 'Neutral', '1': 'Negative', '2': 'Positive'}
    if pd.isna(item):  # 检查是否为NaN
        return item  # 如果是NaN，直接返回
    return label_map.get(str(int(item)), item)

# 转换所有相关列的数字标签
label_columns = ['researcher label', 'ChatGPT label', 'final label']
for column in label_columns:
    if column in data.columns:
        data[column] = data[column].apply(convert_label)

# 保存更新后的 DataFrame
data.to_excel(file_path, index=False)

print(data)

      number                                           sentence  \
0          1  12 U.S. Banks and Tech Giants Targeted with Me...   
1          2  During an international news event today with ...   
2          3  Arjuna Capitals median gender pay gap proposal...   
3          4  In January, Citigroup surprised investors when...   
4          5  In doing so, Citi became the first U.S. compan...   
...      ...                                                ...   
2972       5  "Climate action tops the list of our sustainab...   
2973       6  The goals we set are truly ambitious and will ...   
2974       7  "However, we believe this is the right thing t...   
2975       8  While we will carefully evaluate our options f...   
2976       9  Yum China will continue to enhance transparenc...   

     researcher label ChatGPT label  researcher2 label final label  \
0             Neutral       Neutral                NaN     Neutral   
1             Neutral      Positive                0.0 

In [51]:
data = pd.read_excel('C:\\Users\\ASUS\\Downloads\\news label v1\\news label v1\\Total_data.xlsx', usecols=[1]).values
data2 = pd.read_excel('C:\\Users\\ASUS\\Downloads\\news label v1\\news label v1\\Total_data.xlsx')
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
lst=[]
for x in range(len(data)):
  string = np.array_str(data[x])
  print(string)
  results = nlp(string)
  print(x+1, results)
  string = ''.join(map(str, results))
  if "Positive" in string:
      p='Positive'
      lst.append(p)
  elif "Negative" in string:
      n='Negative'
      lst.append(n)
  elif "Neutral" in string:
      u='Neutral'
      lst.append(u)
for x in range(len(lst)):
    data2.at[x, 'FinBERT_ESGCalls Label'] = lst[x]
df = pd.DataFrame(data2)
df.to_excel('C:\\Users\\ASUS\\Downloads\\news label v1\\news label v1\\Total_data.xlsx', index=False)
df

['12 U.S. Banks and Tech Giants Targeted with Median Gender Pay Gap Shareholder Proposal.']
1 [{'label': 'Neutral', 'score': 0.9964004755020142}]
['During an international news event today with the Equal Rights Advocates and Closing the Womens Wealth Gap Initiative, shareholder advocate Arjuna Capital named the 12 top banks and tech giants it is engaging in 2019 with median gender pay equity resolutions.']
2 [{'label': 'Neutral', 'score': 0.9674502015113831}]
['Arjuna Capitals median gender pay gap proposal has been filed with Adobe, Amazon, American Express, Bank of America, Bank of New York Mellon, Facebook, Google, Intel, JPMorgan, Mastercard and Wells Fargo.']
3 [{'label': 'Neutral', 'score': 0.9976710677146912}]
['In January, Citigroup surprised investors when it revealed that its median or companywide gender pay gap is 29 percent.']
4 [{'label': 'Negative', 'score': 0.4820520579814911}]
['In doing so, Citi became the first U.S. company to respond favorably to a new gender pay sha

,number,sentence,researcher label,ChatGPT label,researcher2 label,final label,BiLSTM Label,BERT Label,FinBERT Label,FinBERT_FinNews Label,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,FinBERT_ESGCalls Label
0,1,12 U.S. Banks and Tech Giants Targeted with Me...,Neutral,Neutral,NaN,Neutral,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral
1,2,During an international news event today with ...,Neutral,Positive,0.0,Neutral,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral
2,3,Arjuna Capitals median gender pay gap proposal...,Neutral,Positive,0.0,Neutral,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral
3,4,"In January, Citigroup surprised investors when...",Neutral,Negative,1.0,Negative,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
4,5,"In doing so, Citi became the first U.S. compan...",Neutral,Positive,2.0,Positive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2972,5,"""Climate action tops the list of our sustainab...",Neutral,Positive,2.0,Positive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral
2973,6,The goals we set are truly ambitious and will ...,Neutral,Positive,0.0,Neutral,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
2974,7,"""However, we believe this is the right thing t...",Positive,Positive,NaN,Positive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral
2975,8,While we will carefully evaluate our options f...,Positive,Positive,NaN,Positive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive


In [45]:
# 确保列名正确无误后，再次尝试打印唯一值
print(data['final label'].unique())  # 根据实际情况调整列名
print(data['FinBERT_ESGCalls Label'].unique())  # 根据实际情况调整列名

['Neutral' 'Negative' 'Positive']
['Neutral' 'Negative' 'Positive']


In [52]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# 讀取 xlsx 檔案
file_path = 'C:\\Users\\ASUS\\Downloads\\news label v1\\news label v1\\Total_data.xlsx'
data = pd.read_excel(file_path)

# 計算不匹配的次數
mismatches = data['final label'] != data['FinBERT_ESGCalls Label']
mismatch_count = mismatches.sum()
total_data = data['final label'] == data['final label']
data_count = total_data.sum()

# 計算準確率
accuracy = accuracy_score(data['final label'], data['FinBERT_ESGCalls Label'])
# 計算精確率
precision = precision_score(data['final label'], data['FinBERT_ESGCalls Label'], average='macro')
# 計算召回率
recall = recall_score(data['final label'], data['FinBERT_ESGCalls Label'], average='macro')
# 計算 F1 分數
f1 = f1_score(data['final label'], data['FinBERT_ESGCalls Label'], average='macro')

print(f"Number of data: {data_count}")
print(f"Number of mismatches: {mismatch_count}")
print(f"Accuracy of the model predictions: {accuracy:.4f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Number of data: 2977
Number of mismatches: 1014
Accuracy of the model predictions: 0.6594
Precision: 0.65
Recall: 0.56
F1 Score: 0.58
